In [8]:
# # -*- coding: utf-8 -*-
"""Airbnb_TopicModel_Visualisation.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/10N8imK1z9b8CkAbl1WAHqYbW1kjZaf16
"""
from Data_Cleaning import datacleaning, datamapping, datapreprocessing
# !pip install pyldavis

import pandas as pd
from nltk.stem import WordNetLemmatizer
import re
# from google.colab import drive

# drive.mount('/content/gdrive')
df_reviews=pd.read_excel('tickets (6).xlsx')
df_reviews

df_reviews.dropna(subset=['Desc'], how='all', inplace=True)

#remove all languages except english
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
words = set(nltk.corpus.words.words())

def keep_english(x):
    englishwords=" ".join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in words or not w.isalpha())
    return englishwords

# df_reviews['Desc']=df_reviews['Desc'].apply(keep_english)
df_reviews['Desc'] = df_reviews['Desc'].map(lambda x: re.sub('[,\.!?]', '', x))
df_reviews['Desc']

import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
nltk.download('stopwords')
# !pip install pyLDAvis
import pyLDAvis
from nltk.corpus import stopwords
stopwords=stopwords.words("english")
stopwords.extend(['from', 'subject', 're', 'edu', 'use', 'infineon', 'camstar','com', 'screenshot','yoda','lot','iptester',\
                 "mkz","mkzsapa","mkzsapaxx","site","xml","number","affect","user","error","etc"])

import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
# lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)
import en_core_web_sm
nlp = en_core_web_sm.load()

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

def gen_words(texts):
    final=[]
    for text in texts:
        new=gensim.utils.simple_preprocess(text,deacc=True)
        final.append(new)
    return (final)
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

fileref = 'lookup.xlsx'
# cleaned = datacleaning(tickets)
processed = datamapping(df_reviews,fileref)
data_words=gen_words(processed['Desc'])
data_words = remove_stopwords(data_words)
lem = WordNetLemmatizer()
for i in range(len(data_words)):
    text_list = data_words[i]
#     text_list = [lancaster.stem(word) for word in text_list]
    text_list = [lem.lemmatize(word) for word in text_list]
    text_list = [lem.lemmatize(word, 'v') for word in text_list]
    text_list = [lem.lemmatize(word, 'a') for word in text_list]
    text_list = [lem.lemmatize(word, 'r') for word in text_list]
    text_list = [lem.lemmatize(word, 's') for word in text_list]
    df_reviews["desc"].loc[i]= " ".join(text_list)


# lemmatized_texts=df_reviews['desc']

#get word tokens

lookup = fileref
typo = pd.read_excel(lookup, sheet_name="typo")
worddict = dict(zip(typo.words, typo.correction))
df_reviews.desc = df_reviews.desc.replace(worddict, regex=True)
data_words=gen_words(df_reviews['desc'])
data_words = remove_stopwords(data_words)

#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

print (data_bigrams_trigrams[0][0:20])

id2word=corpora.Dictionary(data_words)
corpus=[]

for text in data_words:
    new=id2word.doc2bow(tokenize(text))
    corpus.append(new)
    
print(corpus[0][0:20])

# lda_model=gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
#                                          id2word=id2word,
#                                          num_topics=12,
#                                          random_state=101,
#                                          update_every=1,
#                                          chunksize=100,
#                                          passes=10,
#                                          alpha='auto')


# lda_model = gensim.models.lsimodel.LsiModel(
# corpus=corpus, id2word=id2word, num_topics=10,chunksize=100
# )
lda_model = gensim.models.hdpmodel.HdpModel(corpus=corpus, id2word=id2word)

test_doc=corpus[-1]

vector=lda_model[test_doc]
print(vector)


#sort words
def Sort(sub_li):
    sub_li.sort(key=lambda x: x[1])
    sub_li.reverse()
    return (sub_li)

new_vector= Sort(vector)

print(new_vector)

#Visualize
# pyLDAvis.enable_notebook()
# vis=pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word,mds='mmds',R=15)
# vis

[nltk_data] Downloading package wordnet to /ifxhome/manna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /ifxhome/manna/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /ifxhome/manna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


['fajob', 'belong', 'equipment']


NameError: name 'tokenize' is not defined

In [26]:
# df_reviews

In [7]:
lda_model.print_topics()

[(0,
  '0.023*equipment + 0.016*printer + 0.015*server + 0.015*contact + 0.015*area + 0.015*application + 0.015*integration + 0.012*track + 0.009*fajob + 0.009*button'),
 (1,
  '0.006*cannot + 0.006*process + 0.005*equipment + 0.004*track + 0.004*move + 0.004*need + 0.003*state + 0.003*issue + 0.003*mention + 0.003*spec'),
 (2,
  '0.009*test + 0.006*equipment + 0.005*summary + 0.004*miss + 0.004*id + 0.003*till + 0.002*type + 0.002*viewer + 0.002*mode + 0.002*tester'),
 (3,
  '0.004*equipment + 0.004*cannot + 0.003*fajob + 0.002*concern + 0.002*job + 0.002*printer + 0.002*application + 0.002*create + 0.002*fa + 0.002*server'),
 (4,
  '0.004*equipment + 0.004*application + 0.003*test + 0.003*criticl + 0.002*code + 0.002*printer + 0.002*still + 0.002*source + 0.002*client + 0.002*ifxdoc'),
 (5,
  '0.004*equipment + 0.003*object + 0.002*server + 0.002*area + 0.002*application + 0.002*contact + 0.002*printer + 0.002*floor + 0.002*integration + 0.002*ramli'),
 (6,
  '0.003*length + 0.003*eq

In [2]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

ModuleNotFoundError: No module named 'bokeh.util'